# 데이터 크롤링

In [22]:
from selenium import webdriver
import os
import pandas as pd
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from data_to_db import *

wd = webdriver.Chrome()
wd.get('https://stat.molit.go.kr/portal/cate/statMetaView.do?hRsId=58&hFormId=1244&hSelectId=1244&hPoint=00&hAppr=1&hDivEng=&oFileName=&rFileName=&midpath=&sFormId=1244&sStart=2015&sEnd=2023&sStyleNum=562&settingRadio=xlsx')

element = wd.find_element(By.XPATH, '//*[@id="main"]/div/div[3]/div[1]/div/div[2]/div[2]/div/dl/dd[1]/ul/li[1]/a/em')
wd.execute_script("arguments[0].click();", element)

html = wd.page_source
soup = BeautifulSoup(html,'html.parser')

download_folder = os.path.expanduser('~/Downloads')
file_name = soup.select('#main > div > div.contents-wrap > div.contents > div > div:nth-child(2) > div.mu-accordion-body > div > dl > dd:nth-child(2) > ul > li:nth-child(1) > a > em')[0].text[2:]
file_name = file_name.replace(' ','_')
file_path = os.path.join(download_folder, file_name)

try:
    df = pd.read_excel(file_path,sheet_name ='19.연도별 자동차 등록현황',engine ='openpyxl', header = [2])
    print("파일이 성공적으로 읽혔습니다.")
    print(df.head()) 
except FileNotFoundError:
    print("지정한 파일이 존재하지 않습니다.")
except Exception as e:
    print(f"파일을 읽는 중 오류가 발생했습니다: {e}")


파일이 성공적으로 읽혔습니다.
     차종     승용 Unnamed: 2 Unnamed: 3 Unnamed: 4     승합 Unnamed: 6 Unnamed: 7  \
0    년도     관용        자가용        영업용          계     관용        자가용        영업용   
1  2007  20714   11674085     404980   12099779  12650     999807      92492   
2  2008  21388   12025715     436706   12483809  13269     987448      95981   
3  2009  22267   12551833     449719   13023819  14177     967890      98620   
4  2010  22872   13124972     483925   13631769  15039     931740     102946   

  Unnamed: 8     화물  ... Unnamed: 11 Unnamed: 12    특수 Unnamed: 14  \
0          계     관용  ...         영업용           계    관용         자가용   
1    1104949  25230  ...      334584     3171351  2090       10945   
2    1096698  25535  ...      338711     3160338  2110       11372   
3    1080687  25970  ...      341745     3166512  2070       11890   
4    1049725  26306  ...      345805     3203808  2055       12604   

  Unnamed: 15 Unnamed: 16     합계 Unnamed: 18 Unnamed: 19 Unnamed: 20  
0         

# 데이터 전처리

In [23]:
# 10개년만 보려고 했기 때문에 전처리 시행
df = df.loc[7:]

# year데이터만 추출해서 year_id를 생성하고 데이터프레임으로 설정

year_data = df.iloc[:,:1]
year_columns = ['year']
year_data.columns = year_columns

year_data2 = year_data.reset_index(drop=True)
year_data2 = year_data2.reset_index().rename(columns={'index':'year_id'})



In [24]:
# 차종 데이터 프레임 생성을 위해 다른 csv파일을 불러와 전처리

data2 = pd.read_excel(file_path, sheet_name='09.차종별_유형별 현황',engine='openpyxl',header=[2])
data2 = data2.dropna(how='all',axis=1)
data2 = data2.dropna(how='all',axis=0)
data2
kind_data = data2.iloc[1:-1,:1]


In [25]:
# 마지막 데이터를 merge해서 원하는 데이터 프레임 형성

real_kind_list = kind_data['Unnamed: 0'].dropna().tolist()
kind_data=pd.DataFrame(real_kind_list) 
kind_columns = ['kind']
kind_data.columns = kind_columns
kind_data2 = kind_data.reset_index().rename(columns={'index':'kind_id'})

df = df[['차종', 'Unnamed: 4', 'Unnamed: 8', 'Unnamed: 12', 'Unnamed: 16']]
df.columns = ['year', '승용', '승합', '화물', '특수']
df.to_csv('car_data.csv',index=False)
sum_list = []
year_list = []
kind_list = []


for i in range(0, len(df)):
    year_row = df.iloc[i]
    for j in range(len(year_row)):
        if j>0:
            kind_value = year_row.iloc[j]
            year_list.append(i)
            kind_list.append(j-1)
            sum_list.append(kind_value)
        else:
            pass


sum_dict = zip(year_list, kind_list, sum_list)
sum_dict =pd.DataFrame(sum_dict)
column_names = ['year_id','kind_id','total']
sum_dict.columns = column_names
sum_dict = sum_dict.reset_index().rename(columns={'index':'id'})
sum_dict

,id,year_id,kind_id,total
0,0,0,0,15078354
1,1,0,1,970805
2,2,0,2,3285707
3,3,0,3,65998
4,4,1,0,15747171
5,5,1,1,947012
6,6,1,2,3353683
7,7,1,3,70089
8,8,2,0,16561665
9,9,2,1,920320


# 데이터 베이스 생성과 적재

In [26]:
database_name = 'carfirst'
password = '1234'    # 각자 입력!!!

create_database(database_name, password)

Insert_data(year_data2, 'year', 'year_id', database_name, password)
Insert_data(kind_data2, 'kind', 'kind_id', database_name, password)
Insert_data(sum_dict, 'year_kind_sum','id', database_name, password)

In [27]:
# 테이블끼리 fk로 엮기 위해서 코드 실행

connection = get_connection(database_name,password)
with connection.cursor() as cursor:
    sql1 = '''
    ALTER TABLE year_kind_sum
    ADD CONSTRAINT fk_year_id
    FOREIGN KEY (year_id)
    REFERENCES year(year_id)
    '''
    cursor.execute(sql1)

    sql2 = '''
    ALTER TABLE year_kind_sum
    ADD CONSTRAINT fk_kind_id
    FOREIGN KEY (kind_id)
    REFERENCES kind(kind_id)
    '''
    cursor.execute(sql2)